### Get the content of correct answers in the options ###

In [1]:
import pandas as pd
import json


# training data

with open('dataset/MathQA/train.json') as f:
    data = json.loads(f.read())
    
train_df = pd.DataFrame(data)

print('training problems number: ', train_df['Problem'].count())
print(train_df.groupby(['category'])['Problem'].count())

train_df.head(5)

training problems number:  29837
category
gain            5120
general        13273
geometry        2117
other           1814
physics         7063
probability      450
Name: Problem, dtype: int64


,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,the banker ' s gain of a certain sum due 3 yea...,"""explanation : t = 3 years r = 10 % td = ( bg ...","a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...",a,"divide(multiply(const_100, divide(multiply(36,...","multiply(n2,const_100)|multiply(n0,n1)|divide(...",gain
1,average age of students of an adult school is ...,"""explanation : let the original no . of studen...","a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...",d,"multiply(divide(subtract(multiply(add(32, 4), ...","add(n2,n3)|multiply(n1,n2)|multiply(n1,#0)|sub...",general
2,sophia finished 2 / 3 of a book . she calculat...,let xx be the total number of pages in the boo...,"a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281",b,"divide(90, subtract(const_1, divide(2, 3)))","divide(n0,n1)|subtract(const_1,#0)|divide(n2,#1)",general
3,120 is what percent of 50 ?,"""50 * x = 120 - - > x = 2.4 - - > 2.4 expresse...","a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...",b,"multiply(divide(120, 50), const_100)","divide(n0,n1)|multiply(#0,const_100)|",gain
4,there are 10 girls and 20 boys in a classroom ...,"if girls is 10 and boys is 20 , then 10 / 20 ....","a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...",a,"divide(10, 20)","divide(n0,n1)",other


In [2]:
print(train_df['options'][:5])
print(train_df.iloc[10])
print(len(train_df))

0    a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...
1    a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...
2      a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281
3    a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...
4    a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...
Name: options, dtype: object
Problem              the average weight of 8 person ' s increases b...
Rationale            "total weight increased = ( 8 x 1.5 ) kg = 6 k...
options              a ) 81 kg , b ) 85 kg , c ) 90 kg , d ) 100 kg...
correct                                                              a
annotated_formula                            add(multiply(8, 1.5), 75)
linear_formula                             multiply(n0,n1)|add(n2,#0)|
category                                                       general
Name: 10, dtype: object
29837


In [3]:
# validation data

with open('dataset/MathQA/dev.json') as f:
    data = json.loads(f.read())
    
val_df = pd.DataFrame(data)

print('validation problems number: ', val_df['Problem'].count())
print(val_df.groupby(['category'])['Problem'].count())

val_df.head(5)

validation problems number:  4475
category
gain            743
general        2006
geometry        308
other           288
physics        1056
probability      74
Name: Problem, dtype: int64


,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,a multiple choice test consists of 4 questions...,"""5 choices for each of the 4 questions , thus ...","a ) 24 , b ) 120 , c ) 625 , d ) 720 , e ) 1024",c,"power(5, 4)","power(n1,n0)|",general
1,a 3 - digit positive integer is chosen at rand...,p ( three odd digits ) = 5 / 9 * 5 / 10 * 5 / ...,"a ) 1 / 2 , b ) 31 / 36 , c ) 49 / 54 , d ) 7 ...",b,"subtract(const_1, power(divide(const_1, const_...","divide(const_1,const_2)|power(#0,n0)|subtract(...",general
2,if x and y are positive integers and 7 + x + y...,"""try each answer choices . for a : 7 + 3 + xy ...","a ) 3 , b ) 5 , c ) 6 , d ) 8 , e ) 9",c,"add(21, const_1)","add(n1,const_1)|",general
3,the hcf and lcm of two numbers m and n are res...,"answer we have , m x n = 6 x 210 = 1260 ∴ 1 / ...","a ) 1 / 35 , b ) 3 / 35 , c ) 5 / 37 , d ) 2 /...",d,"divide(72, multiply(6, 210))","multiply(n0,n1)|divide(n2,#0)",general
4,"in a kilometer race , a beats b by 48 meters o...","""time taken by b run 1000 meters = ( 1000 * 12...","a ) 238 sec , b ) 190 sec , c ) 667 sec , d ) ...",a,"subtract(divide(multiply(const_1, const_1000),...","divide(n0,n1)|multiply(const_1,const_1000)|div...",physics


#### 1、normalization 2/3 10%
#### 2、tokens（word + number）
#### 3、regression

In [4]:
import re

def replace_fraction(sen):
    # replace all string like "2 / 3 "
    pattern = re.compile(r'[0-9]+\s\/\s[0-9]+\s')    
    results = pattern.findall(sen)
    
    if results != None:
        new_results = []
        for i in results:
            nums = i.split(' / ')
            new_results.append(str(int(nums[0])/int(nums[1]))[:8]+' ')
        for i in range(len(results)):
            sen = sen.replace(results[i], new_results[i])
    return sen

def replace_percentage(sen):
    # replace all string like "20 %"
    pattern = re.compile(r'[0-9]+\.?[0-9]+\s%')   
    results = pattern.findall(sen)
    
    if results != None:
#         print(results)
        new_results = []
        for i in results:
            nums = i.split(' ')
            new_results.append(str(float(nums[0])/100))
        for i in range(len(results)):
            sen = sen.replace(results[i], new_results[i])
    return sen
    
def replace_punc(sen):
    sen = re.sub(r'\s[^\w\s]\s', ' ', sen)
    return sen


def problem_procession(problem):
    problem = problem.apply(replace_fraction)
    problem = problem.apply(replace_percentage)
    problem = problem.apply(replace_punc)
    return problem

In [5]:
train_problems = problem_procession(train_df.Problem)
print(train_problems.head(5))

val_problems = problem_procession(val_df.Problem)
print(val_problems.head(5))

0    the banker s gain of a certain sum due 3 years...
1    average age of students of an adult school is ...
2    sophia finished 0.666666 of a book she calcula...
3                          120 is what percent of 50 ?
4    there are 10 girls and 20 boys in a classroom ...
Name: Problem, dtype: object
0    a multiple choice test consists of 4 questions...
1    a 3 digit positive integer is chosen at random...
2    if x and y are positive integers and 7 x y xy ...
3    the hcf and lcm of two numbers m and n are res...
4    in a kilometer race a beats b by 48 meters or ...
Name: Problem, dtype: object


In [6]:
def separate_numbers(sen):    
    pattarn = re.compile(r'\s[0-9]+\.?[0-9]*\s')
    numbers = pattarn.findall(sen)
    numbers = [i[1:-1] for i in numbers]
    return numbers
    
def separate_text(sen):
    pattarn = re.compile(r'\s[0-9]+\.?[0-9]*\s')
    numbers = pattarn.findall(sen)
    for i in range(len(numbers)):
        sen = sen.replace(numbers[i], " ")  
    return sen    

sen = "the banker ' s gain of a 0.66 certain 3.2549 sum due 3 years 21.5 % hence at 10 % per annum is rs . 36 . what is the present worth ?"
print(separate_numbers(sen))
print(separate_text(sen))

['0.66', '3.2549', '3', '21.5', '10', '36']
the banker ' s gain of a certain sum due years % hence at % per annum is rs . . what is the present worth ?


In [7]:
text = train_problems.apply(separate_text)
print(text)

0        the banker s gain of a certain sum due years h...
1        average age of students of an adult school is ...
2        sophia finished of a book she calculated that ...
3                                 120 is what percent of ?
4        there are girls and boys in a classroom what i...
                               ...                        
29832    a man invests some money partly in stock at an...
29833    average of matches is how many runs one should...
29834                          solve below question x - 23
29835                                      calculate ? 352
29836    7.51 7.86 8.09 8.30 7.73 7.96 a vending machin...
Name: Problem, Length: 29837, dtype: object


In [8]:
numbers = train_problems.apply(separate_numbers)
print(numbers)

0                                             [3, 0.1, 36]
1                                         [40, 120, 32, 4]
2                                           [0.666666, 90]
3                                                     [50]
4                                                 [10, 20]
                               ...                        
29832                                     [0.1, 90, 5, 90]
29833                                          [10, 32, 6]
29834                                               [2, 1]
29835                                              [85184]
29836    [8.22, 8.36, 7.83, 8.01, 8.25, 8.53, 8, 1000, ...
Name: Problem, Length: 29837, dtype: object


In [9]:
max_token = 0
for t in text:
    if len(t) > max_token:
        max_token = len(t)
print("the max tokens in the texts: ", max_token)

the max tokens in the texts:  745


### normalize the answers ###

In [10]:
answers = list()

for i in range(len(train_df)):
    options = train_df.iloc[i]['options']
    tab = train_df.iloc[i]['correct']
    index_s = options.find(tab)+4
    index_len = options[index_s:].find(',')
    ans = options[index_s:index_s+index_len]
    answers.append(ans)
    
answers = pd.Series(answers)
print(answers.head(10))

answers = answers.apply(replace_fraction)
answers = answers.apply(replace_percentage)

print(answers.head(10))

0    rs . 400 
1         240 
2         270 
3       240 % 
4       1 / 2 
5         122 
6      21.5 % 
7           3 
8        6.25 
9          80 
dtype: object
0    rs . 400 
1         240 
2         270 
3         2.4 
4         0.5 
5         122 
6       0.215 
7           3 
8        6.25 
9          80 
dtype: object


### processing text ###

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

max_features = 5000
embedding_dim = 128
sequence_length = 150

# build a word vocab that contains top 5000 most frequent words
tok = Tokenizer(num_words=max_features)
tok.fit_on_texts(text)

print("token vocab length:", len(tok.word_index))
# print(tok.word_index)

x_train_seq = tok.texts_to_sequences(text)
max_seq = 0
for i in x_train_seq:
    if len(i) > max_seq:
        max_seq = len(i)
print("max sequence length: ", max_seq)

x_train = sequence.pad_sequences(x_train_seq, maxlen=sequence_length, padding='post')

print(x_train)
print(x_train.shape)

# print(tok.texts_to_sequences(train_df.Problem))

token vocab length: 8655
max sequence length:  137
[[   1 1382   42 ...    0    0    0]
 [  28  104    2 ...    0    0    0]
 [ 760    2    3 ...    0    0    0]
 ...
 [1131  719  640 ...    0    0    0]
 [ 619    0    0 ...    0    0    0]
 [ 363 4743  363 ...    0    0    0]]
(29837, 150)


### processing numbers ###

In [12]:
max_number_seq = 0
index = 0
for i in numbers:
    if len(i) > max_number_seq:
        max_number_seq = len(i)
print("max number sequence length: ", max_number_seq)
print(numbers)

x_train_num = sequence.pad_sequences(numbers, dtype='float32', truncating='post', padding='post')
print(x_train_num)
print(x_train_num.shape)

max number sequence length:  21
0                                             [3, 0.1, 36]
1                                         [40, 120, 32, 4]
2                                           [0.666666, 90]
3                                                     [50]
4                                                 [10, 20]
                               ...                        
29832                                     [0.1, 90, 5, 90]
29833                                          [10, 32, 6]
29834                                               [2, 1]
29835                                              [85184]
29836    [8.22, 8.36, 7.83, 8.01, 8.25, 8.53, 8, 1000, ...
Name: Problem, Length: 29837, dtype: object
[[3.00000e+00 1.00000e-01 3.60000e+01 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [4.00000e+01 1.20000e+02 3.20000e+01 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [6.66666e-01 9.00000e+01 0.00000e+00 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 ...
 [2.00000e+00 1.00000e+00 0

In [13]:
print(x_train.shape)
print(x_train_num.shape)

(29837, 150)
(29837, 21)


### processing answers ###

In [14]:

# build a word vocab that contains top  most frequent words
tok = Tokenizer()
tok.fit_on_texts(answers)

print("token vocab length:", len(tok.word_index))
# print(tok.word_index)

y_train_seq = tok.texts_to_sequences(answers)
max_seq = 0
for i in y_train_seq:
    if len(i) > max_seq:
        max_seq = len(i)
print("max sequence length: ", max_seq)

y_train = sequence.pad_sequences(y_train_seq, padding='post')

print(y_train)
print(y_train.shape)

token vocab length: 3063
max sequence length:  9
[[   9   64    0 ...    0    0    0]
 [  84    0    0 ...    0    0    0]
 [ 146    0    0 ...    0    0    0]
 ...
 [  11    0    0 ...    0    0    0]
 [ 597    0    0 ...    0    0    0]
 [  63 3063    0 ...    0    0    0]]
(29837, 9)


### building model ###

In [15]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import layers

# model = tf.keras.Sequential()

# model.add(
#     layers.Embedding(
#         input_dim=max_features,        # the size of vocabulary
#         input_length=sequence_length,     # Length of input sequences
#         output_dim=32,        # dimension of the dense embedding
#     )
# )

# model.add(
#     layers.LSTM(
#         unit=32,
#     )
# )

# model.add(layers.Dropout(0.1))
# model.add(layers.Dense(units=256, activation='relu'))

# model.add(layers.Dropout(0.1))
# model.add(layers.Dense(units=1, activation='sigmoid'))

# model.compile(
#     loss='sparse_categorical_crossentropy',
#     optimizer='adam',
#     metrics=['accuracy']
# )


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(sequence_length,), dtype="int64")

# input_dim: the size of vocabulary
# input_length: Length of input sequences
# output_dim: dimension of the dense embedding

x = layers.Embedding(
    input_dim=max_features, 
    output_dim=embedding_dim,
    input_length=sequence_length, 
)(inputs)

x = layers.LSTM(units=150, activation='relu',)(x)

x = layers.Dropout(0.1)(x)
x = layers.Dense(units=150, activation='relu')(x)

x = layers.Concatenate(axis=1)([x, x_train_num])
x = layers.Dense(units=171, activation='linear')(x)
# len(x_train) + len(x_train_num) = 150 + 21 = 171

predictions = layers.Dense(9, activation='linear')(x)

model = tf.keras.Model(inputs, predictions)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 128)          640000    
_________________________________________________________________
lstm (LSTM)                  (None, 150)               167400    
_________________________________________________________________
dropout (Dropout)            (None, 150)               0         
_________________________________________________________________
dense (Dense)                (None, 150)               22650     
_________________________________________________________________
concatenate (Concatenate)    (29837, 171)              0         
_________________________________________________________________
dense_1 (Dense)              (29837, 171)              29412 

In [17]:
print(x_train.shape)
print(x_train_num.shape)
print(y_train.shape)


(29837, 150)
(29837, 21)
(29837, 9)


In [18]:
epochs = 3

model.fit(
    x=x_train,
    y=y_train,
#     batch_size=29837,
#     validation_split=0.2,
    epochs=epochs,
)

Epoch 1/3


InvalidArgumentError:  Incompatible shapes: [29837,150] vs. [32,150]
	 [[node model/dropout/dropout/Mul_1 (defined at C:\Users\Liang\AppData\Local\Temp/ipykernel_17268/1007158877.py:8) ]] [Op:__inference_train_function_2305]

Function call stack:
train_function


In [ ]:
# from tensorflow.keras.layers import TextVectorization

# def custom_standardization(input_data):
#     lowercase = tf.strings.lower(input_data)
# #     stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
#     return lowercase



# vectorize_layer = TextVectorization(
#     standardize = custom_standardization,
#     max_tokens=max_features,
#     output_mode="int",
#     output_sequence_length=sequence_length,
# )

In [ ]:
# def vectorize_text(text):
#     text = tf.expand_dims(text, -1)
#     return vectorize_layer(text)
# #     return tok.texts_to_sequences(text)


# # Vectorize the data.
# train_ds =text.map(vectorize_text)
# # val_ds = raw_val_ds.map(vectorize_text)
# # test_ds = raw_test_ds.map(vectorize_text)

# # Do async prefetching / buffering of the data for best performance on GPU.
# train_ds = train_ds.cache().prefetch(buffer_size=10)
# # val_ds = val_ds.cache().prefetch(buffer_size=10)
# # test_ds = test_ds.cache().prefetch(buffer_size=10)